# Example notebook with a FreePDK45 inverter

This needs to have the pdkmaster code installed in the environment where this notebook is executed.

In [ ]:
from matplotlib import pyplot, ticker

from pdkmaster.design import layout as lay, circuit as ckt
from pdkmaster.techs.freepdk45 import tech, layoutfab, cktfab

nmos = tech.primitives.nmos_vtg
pmos = tech.primitives.pmos_vtg

In [ ]:
# TODO: this should be defined by the technology
plotting = lay.Plotter({
    "pwell": {"fc": (1.0, 1.0, 0.0, 0.2), "ec": "orange", "zorder": 10},
    "nwell": {"fc": (0.0, 0.0, 0.0, 0.1), "ec": "grey", "zorder": 10},
    "active": {"fc": "lawngreen", "ec": "lawngreen", "zorder": 11},
    "poly": {"fc": "red", "ec": "red", "zorder": 12},
    "nimplant": {"fc": "purple", "ec": "purple", "alpha": 0.3, "zorder": 13},
    "pimplant": {"fc": "blueviolet", "ec": "blueviolet", "alpha": 0.3, "zorder": 13},
    "vthg": {"fc": (0.0, 0.0, 0.0, 0.0), "ec": "grey", "zorder": 13},
    "vthl": {"fc": (1, 1, 1, 0.3), "ec": "whitesmoke", "zorder": 13},
    "vthh": {"fc": (0.0, 0.0, 0.0, 0.2), "ec": "dimgrey", "zorder": 13},
})

In [ ]:
l_n = nmos.computed.min_l
l_p = pmos.computed.min_l
w_n = nmos.computed.min_w
w_p = 2*w_n

# TODO: Use packer to derive minimal placement of p transistor
impl_n_enc = nmos.min_gateimplant_enclosure[0] # This is not generic yet
impl_p_enc = pmos.min_gateimplant_enclosure[0] # This is not generic yet
y_p = 0.5*w_n + impl_n_enc + impl_p_enc + 0.5*w_p

# Create the circuit
inv = cktfab.new_circuit("inverter")
n1 = inv.new_instance("n1", nmos, l=l_n, w=w_n)
p1 = inv.new_instance("p1", pmos, l=l_p, w=w_p)
in_ = inv.new_net("IN", external=True)
out = inv.new_net("OUT", external=True)
vdd = inv.new_net("VDD", external=True)
gnd = inv.new_net("GND", external=True)
in_.childports += (n1.ports.gate, p1.ports.gate)
out.childports += (n1.ports.sourcedrain1, p1.ports.sourcedrain1)
gnd.childports += (n1.ports.sourcedrain2, n1.ports.bulk)
vdd.childports += (p1.ports.sourcedrain2, p1.ports.bulk)

# Create layout
placer = ckt.CircuitLayouter(inv)
placer.place(n1, x=0.0, y=0.0)
placer.place(p1, x=0.0, y=y_p)

pyplot.figure(figsize=(4.0, 7.0))

plotting.plot(inv.layout)

ax = pyplot.gca()
ax.set_title("inverter")
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.axis("square")
ax.axis("equal")
ax.axis([-0.2, 0.2, -0.2, 0.5])
ax.grid("on")

pyplot.show()